### ***Problme02***
---

In [16]:
### Import library & data set
# library
from google.colab import drive
drive.mount("/content/drive")
import sys
import numpy as np
import pandas as pd
from collections import Counter
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# pytorch
import torch
from torch import nn
import tensorflow as tf
from tensorflow import keras
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from tensorflow.keras import optimizers
from torchvision.transforms import ToTensor
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
# keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import optimizers
from keras.preprocessing import sequence, text
from keras.preprocessing.text import Tokenizer

# dataset
# mnist 28x28
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor())

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor())
# 20ng
categories =  ['alt.atheism', 'talk.religion.misc', 'sci.space']
ng = fetch_20newsgroups(categories=categories)
vectorizer = TfidfVectorizer()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### mnist
# setting
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("gpu available:", device)

# dataloader
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# mlp
class mlp(nn.Module):
  def __init__(self):
    super(mlp, self).__init__()
    self.fc1 = nn.Linear(28*28, 50)
    self.fc1_drop = nn.Dropout(0.2)
    self.fc2 = nn.Linear(50, 50)
    self.fc2_drop = nn.Dropout(0.2)
    self.fc3 = nn.Linear(50, 10)

  def forward(self, x):
    x = x.view(-1, 28*28)
    x = F.relu(self.fc1(x))
    x = self.fc1_drop(x)
    x = F.relu(self.fc2(x))
    x = self.fc2_drop(x)
    return F.log_softmax(self.fc3(x), dim=1)
model = mlp().to(device)
print(model)

gpu available: cpu
mlp(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc1_drop): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (fc2_drop): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
)


In [ ]:
# hyperparameters
learning_rate = 0.01
batch_size = 64
epochs = 3

# compile
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# implement
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.289609  [   63/60000]
loss: 0.418753  [ 6363/60000]
loss: 0.687011  [12663/60000]
loss: 0.199743  [18963/60000]
loss: 0.462815  [25263/60000]
loss: 0.211124  [31563/60000]
loss: 0.532388  [37863/60000]
loss: 0.344739  [44163/60000]
loss: 0.669365  [50463/60000]
loss: 0.242764  [56763/60000]
Test Error: 
 Accuracy: 90.5%, Avg loss: 0.343679 

Epoch 2
-------------------------------
loss: 0.325420  [   63/60000]
loss: 0.175350  [ 6363/60000]
loss: 0.525966  [12663/60000]
loss: 0.194495  [18963/60000]
loss: 0.410894  [25263/60000]
loss: 0.266307  [31563/60000]
loss: 0.569222  [37863/60000]
loss: 0.506687  [44163/60000]
loss: 0.497551  [50463/60000]
loss: 0.311953  [56763/60000]
Test Error: 
 Accuracy: 91.6%, Avg loss: 0.327195 

Epoch 3
-------------------------------
loss: 0.162931  [   63/60000]
loss: 0.315540  [ 6363/60000]
loss: 0.449689  [12663/60000]
loss: 0.210662  [18963/60000]
loss: 0.474274  [25263/60000]
loss: 0.218274  [31563/600

In [49]:
### 20ng
# data split
X = ng.data
y = to_categorical(ng.target)
t = Tokenizer()
t.fit_on_texts(X)
X = t.texts_to_matrix(X, mode='count')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

# mlp
model = Sequential()
model.add(Flatten(input_shape=(27956,)))
model.add(Dense(500, activation="relu"))
model.add(Dense(250, activation="relu"))
model.add(Dense(3, activation="softmax"))
model.summary()

# compile and fit
opt = optimizers.Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy",optimizer = opt, metrics=["accuracy"])
model.fit(X_train, y_train, batch_size=64, epochs=3, shuffle=True)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_11 (Flatten)        (None, 27956)             0         
                                                                 
 dense_33 (Dense)            (None, 500)               13978500  
                                                                 
 dense_34 (Dense)            (None, 250)               125250    
                                                                 
 dense_35 (Dense)            (None, 3)                 753       
                                                                 
Total params: 14,104,503
Trainable params: 14,104,503
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
16/16 [==============================] - 4s 187ms/step - loss: 3.6125 - accuracy: 0.6169
Epoch 2/3
16/16 [==============================] - 5s 289ms/step - loss: 0

In [50]:
# eval
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', round(loss,2))
print('Test accuracy:', round(accuracy,2))

15/15 [==============================] - 1s 52ms/step - loss: 0.5282 - accuracy: 0.9061
Test loss: 0.53
Test accuracy: 0.91
